In [1]:
from IPython.display import display, HTML, Audio
from datasets import load_dataset,concatenate_datasets
from datasets import ClassLabel, load_dataset, load_metric
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
import os
import glob
import numpy as np
import torchaudio
import random
import pykakasi 
import re
import MeCab
import random
import pandas as pd
import json
import transformers
print(transformers.__version__)


vocabPath="./vocab_jp_hiragana_1027.json"
model_temp_output_dir="./wav2vec2-large-xlsr-jp-test1027_hiragana1"

4.11.2


In [2]:
#common vocie
datasets_common_voice_train = load_dataset("common_voice", "ja", split="train+validation")
datasets_common_voice_test = load_dataset("common_voice", "ja", split="test")

datasets_common_voice_train = datasets_common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
datasets_common_voice_test = datasets_common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

print(datasets_common_voice_train)
print(datasets_common_voice_train[0])

print(datasets_common_voice_test)
print(datasets_common_voice_test[0])

Reusing dataset common_voice (/home/user/.cache/huggingface/datasets/common_voice/ja/6.1.0/078d412587e9efeb0ae2e574da99c31e18844c496008d53dc5c60f4159ed639b)
Reusing dataset common_voice (/home/user/.cache/huggingface/datasets/common_voice/ja/6.1.0/078d412587e9efeb0ae2e574da99c31e18844c496008d53dc5c60f4159ed639b)


Dataset({
    features: ['path', 'sentence'],
    num_rows: 1308
})
{'path': '/home/user/.cache/huggingface/datasets/downloads/extracted/68d8e1741c324687f1826c19c295d9e410c7903d8cd5f07acdbc399e28078cd9/cv-corpus-6.1-2020-12-11/ja/clips/common_voice_ja_19817895.mp3', 'sentence': '予想外の事態に、電力会社も、ちょっぴり困惑気味だ。'}
Dataset({
    features: ['path', 'sentence'],
    num_rows: 632
})
{'path': '/home/user/.cache/huggingface/datasets/downloads/extracted/68d8e1741c324687f1826c19c295d9e410c7903d8cd5f07acdbc399e28078cd9/cv-corpus-6.1-2020-12-11/ja/clips/common_voice_ja_21409740.mp3', 'sentence': '祖母は、おおむね機嫌よく、サイコロをころがしている。'}


In [3]:
# TEDxJP-10K 
datasets_TEDxJP = load_dataset('csv', data_files="TEDxJP.txt",delimiter="|",column_names=["path","sentence"])["train"]

#skip some row that contain special char
datasets_TEDxJP=datasets_TEDxJP.filter(lambda batch: not re.search(r'[：×ⅿ＋％㎏＝繫]', batch["sentence"]))  

print(datasets_TEDxJP)
print(datasets_TEDxJP[0])

Using custom data configuration default-56b4f930f29339e2
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-56b4f930f29339e2/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-56b4f930f29339e2/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-24f8b3f842972061.arrow


Dataset({
    features: ['path', 'sentence'],
    num_rows: 9880
})
{'path': './TEDxJP/-6K2nN9aWsg-00002686-00002940.wav', 'sentence': '今大学教員をやってるんですけど'}


In [4]:
#CSS10
datasets_CSS10 = load_dataset('csv', data_files="ja/transcript.txt",delimiter="|",column_names=["path","sentence","pronounciation","time"])["train"]
datasets_CSS10=datasets_CSS10.map(lambda batch: {'path': "ja/"+batch['path']}, remove_columns=["pronounciation","time"])

datasets_CSS10=datasets_CSS10.filter(lambda batch: bool(batch["sentence"]))   #remove no text
datasets_CSS10=datasets_CSS10.filter(lambda batch: not re.search('[\(\)\（\）鐉]', str(batch["sentence"])))   #remove specical char

print(datasets_CSS10)
print(datasets_CSS10[0])

Using custom data configuration default-28bdfa812000a5a9
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-28bdfa812000a5a9/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-28bdfa812000a5a9/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-6ad16e126203c312.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-28bdfa812000a5a9/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-de4479a1c56c1526.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-28bdfa812000a5a9/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-92245ac76b5ccf0c.arrow


Dataset({
    features: ['path', 'sentence'],
    num_rows: 6832
})
{'path': 'ja/meian/meian_0000.wav', 'sentence': 'この前探った時は、途中に瘢痕の隆起があったので、ついそこが行きどまりだとばかり思って、ああ云ったんですが、'}


In [5]:
#jsut
transcript_utf8_list=glob.glob("jsut_ver1.1/*/transcript_utf8.txt")

datasets_JSUT_list=[]
for csvPath in transcript_utf8_list:
    datasets = load_dataset('csv', data_files=csvPath,delimiter=":",column_names=["path","sentence"])["train"]
    datasets = datasets.map(lambda batch: {'path': os.path.dirname(csvPath)+"/wav/"+batch['path']+".wav"})
    datasets_JSUT_list+=[datasets]

datasets_JSUT=concatenate_datasets(datasets_JSUT_list)
datasets_JSUT=datasets_JSUT.filter(lambda batch: not re.search(r'[Ｉ\d剝禕]', batch["sentence"]))   #remove one that include digit  

print(datasets_JSUT)
print(datasets_JSUT[0])

Using custom data configuration default-f16ffcc23e353dc4
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-f16ffcc23e353dc4/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-f16ffcc23e353dc4/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-1d01abc3fffd7b41.arrow
Using custom data configuration default-a38cc6a8d6293a53
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-a38cc6a8d6293a53/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-a38cc6a8d6293a53/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-4b66c91fe7ce2b90.arrow
Using custom data configuration default-17dbc6fe17773a33
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-17dbc6fe17773a33/0.0.0/9144

Dataset({
    features: ['path', 'sentence'],
    num_rows: 7260
})
{'path': 'jsut_ver1.1/onomatopee300/wav/ONOMATOPEE300_001.wav', 'sentence': '目がちかちかするのだが、なにかの病気だろうか？'}


In [6]:
transcript_utf8_list=glob.glob("jvs_ver1/*/*/*.txt")

datasets_jvs_list=[]
for csvPath in transcript_utf8_list:
    datasets = load_dataset('csv', data_files=csvPath,delimiter=":",column_names=["path","sentence"])["train"]
    datasets = datasets.map(lambda batch: {'path': os.path.dirname(csvPath)+"/wav24kHz16bit/"+batch['path']+".wav"})
    datasets_jvs_list+=[datasets]

datasets_jvs=concatenate_datasets(datasets_jvs_list)
datasets_jvs=datasets_jvs.filter(lambda batch: not re.search(r'[Ｉ\d]', batch["sentence"]))   #remove one that include digit 
datasets_jvs=datasets_jvs.filter(lambda batch: os.path.exists(batch["path"]))   #remove no file

print(datasets_jvs)
print(datasets_jvs[0])

Using custom data configuration default-2b3d2bcfcb0279a0
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-2b3d2bcfcb0279a0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-2b3d2bcfcb0279a0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-07eb12e0671fcb38.arrow
Using custom data configuration default-dc587e8523ada2ce
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-dc587e8523ada2ce/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-dc587e8523ada2ce/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-cb0c571c953abd09.arrow
Using custom data configuration default-4d335ef6f0aa73be
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-4d335ef6f0aa73be/0.0.0/9144

Dataset({
    features: ['path', 'sentence'],
    num_rows: 14557
})
{'path': 'jvs_ver1/jvs050/whisper10/wav24kHz16bit/VOICEACTRESS100_001.wav', 'sentence': 'また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配されることも多い。'}


In [7]:
#jsss

csvPath="preprocess_jsss_ver1.txt"
datasetsAudioSplit = load_dataset('csv', data_files=csvPath,delimiter="\t",column_names=["path","sentence"])["train"]    

transcript_utf8_list=glob.glob("jsss_ver1/short-form/*/transcript_utf8.txt")
transcript_utf8_list+=["jsss_ver1/simplification/transcript_utf8.txt"]

datasets_jsss_list=[]
for csvPath in transcript_utf8_list:
    datasets = load_dataset('csv', data_files=csvPath,delimiter=":",column_names=["path","sentence"])["train"]
    datasets = datasets.map(lambda batch: {'path': os.path.dirname(csvPath)+"/wav24kHz16bit/"+batch['path']+".wav"})
    datasets_jsss_list+=[datasets]
    
    
    
datasets_jsss_list+=[datasetsAudioSplit]
datasets_jsss_list=concatenate_datasets(datasets_jsss_list)
datasets_jsss_list



datasets_jsss_list=datasets_jsss_list.filter(lambda batch: not re.search(r'[a-zA-Z\d]', batch["sentence"]))   #remove one that include digit 
datasets_jsss_list=datasets_jsss_list.filter(lambda batch: os.path.exists(batch["path"]))   #remove no file

Using custom data configuration default-813dbe768c4d67cc
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-813dbe768c4d67cc/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Using custom data configuration default-05197c17428d7ec6
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-05197c17428d7ec6/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-05197c17428d7ec6/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-e2783b69df95730f.arrow
Using custom data configuration default-25ba3cd861756fbc
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-25ba3cd861756fbc/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-25ba3cd861756fbc/0.0.0/9144e0a4e8435090117cea53e6c75371

In [8]:
for i in [datasets_common_voice_train,datasets_common_voice_test,datasets_TEDxJP,datasets_CSS10,datasets_JSUT,datasets_jvs,datasets_jsss_list]:
    print(i)
    print(i[0])
    print("======================")


Dataset({
    features: ['path', 'sentence'],
    num_rows: 1308
})
{'path': '/home/user/.cache/huggingface/datasets/downloads/extracted/68d8e1741c324687f1826c19c295d9e410c7903d8cd5f07acdbc399e28078cd9/cv-corpus-6.1-2020-12-11/ja/clips/common_voice_ja_19817895.mp3', 'sentence': '予想外の事態に、電力会社も、ちょっぴり困惑気味だ。'}
Dataset({
    features: ['path', 'sentence'],
    num_rows: 632
})
{'path': '/home/user/.cache/huggingface/datasets/downloads/extracted/68d8e1741c324687f1826c19c295d9e410c7903d8cd5f07acdbc399e28078cd9/cv-corpus-6.1-2020-12-11/ja/clips/common_voice_ja_21409740.mp3', 'sentence': '祖母は、おおむね機嫌よく、サイコロをころがしている。'}
Dataset({
    features: ['path', 'sentence'],
    num_rows: 9880
})
{'path': './TEDxJP/-6K2nN9aWsg-00002686-00002940.wav', 'sentence': '今大学教員をやってるんですけど'}
Dataset({
    features: ['path', 'sentence'],
    num_rows: 6832
})
{'path': 'ja/meian/meian_0000.wav', 'sentence': 'この前探った時は、途中に瘢痕の隆起があったので、ついそこが行きどまりだとばかり思って、ああ云ったんですが、'}
Dataset({
    features: ['path', 'sentence'],
    num_row

In [ ]:
#create vocab

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\‘\”\�‘、。．！，・―─~｢｣『』〆｡\\\\※\[\]\{\}「」〇？…]'
wakati = MeCab.Tagger("-Owakati")
kakasi = pykakasi.kakasi()
kakasi.setMode("J","H")      # kanji to hiragana
kakasi.setMode("K","H")      # katakana to hiragana
conv = kakasi.getConverter()

FULLWIDTH_TO_HALFWIDTH = str.maketrans(
    '　０１２３４５６７８９ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ！゛＃＄％＆（）＊＋、ー。／：；〈＝〉？＠［］＾＿‘｛｜｝～',
    ' 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&()*+,-./:;<=>?@[]^_`{|}~',
)
def fullwidth_to_halfwidth(s):
    return s.translate(FULLWIDTH_TO_HALFWIDTH)



def preprocess_text(batch):
    batch["sentence"] = fullwidth_to_halfwidth(batch["sentence"])
    batch["sentence"] = re.sub(chars_to_ignore_regex,' ', batch["sentence"]).lower()  #remove special char
    batch["sentence"] = wakati.parse(batch["sentence"])                              #add space
    batch["sentence"] = conv.do(batch["sentence"])                                   #covert to hiragana
    # batch["sentence"] = " ".join(batch["sentence"].split())+" "                    #remove multiple space    
    batch["sentence"] = " ".join(batch["sentence"].split())+" "                    #remove multiple space    
    return batch


def createVocabList(dataset):
    def extract_all_chars(batch):
        all_text = " ".join(batch["sentence"])
        vocab = list(set(all_text))
        return {"vocab": [vocab], "all_text": [all_text]}

    
    dataset=dataset.map(preprocess_text)
    vocab_train = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names) #concat all text
    vocab_list = list(set(vocab_train["vocab"][0]) )      #convert to set
    vocab_list = sorted(vocab_list)
    vocab_dict = {v: k for k, v in enumerate(vocab_list)}   # convert to dict
    vocab_dict["|"] = vocab_dict[" "]
    del vocab_dict[" "]
    vocab_dict["[UNK]"] = len(vocab_dict)
    vocab_dict["[PAD]"] = len(vocab_dict)

    with open(vocabPath, 'w')as vocab_file:
        json.dump(vocab_dict, vocab_file)

    print(vocab_dict)
    
    
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = torchaudio.functional.resample(speech_array, sampling_rate, 16000)[0].numpy()
    batch["target_text"] = batch["sentence"]
    return batch


In [ ]:
    
# create vocab
createVocabList(concatenate_datasets([datasets_common_voice_train,datasets_common_voice_test,datasets_TEDxJP,datasets_CSS10,datasets_JSUT,datasets_jvs,datasets_jsss_list]))




In [11]:



# concat train
datasets_train=concatenate_datasets([datasets_common_voice_train,datasets_TEDxJP,datasets_CSS10,datasets_JSUT,datasets_jvs,datasets_jsss_list])
# datasets_train=concatenate_datasets([datasets_common_voice_train])
datasets_test=datasets_common_voice_test

#preprocess
datasets_train=datasets_train.map(preprocess_text)  # preprocess text
datasets_train=datasets_train.filter(lambda batch: bool(batch["sentence"]))   #remove empty
datasets_test=datasets_test.map(preprocess_text)  # preprocess text



  0%|          | 0/43917 [00:00<?, ?ex/s]

/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.


  0%|          | 0/44 [00:00<?, ?ba/s]

  0%|          | 0/632 [00:00<?, ?ex/s]

In [12]:
#filter long audio, prevent cuda out of memory
def filterLen(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    return speech_array.shape[1]/sampling_rate<11.0


datasets_train=datasets_train.filter(filterLen,num_proc=14,)
datasets_train

Dataset({
    features: ['path', 'sentence'],
    num_rows: 42403
})

In [13]:

datasets_train = datasets_train.map(speech_file_to_array_fn)
datasets_test = datasets_test.map(speech_file_to_array_fn) #num_proc=4



print(datasets_train[0]["target_text"])
Audio(data=np.asanyarray(datasets_train[0]["speech"]), autoplay=False, rate=16000)

  0%|          | 0/42403 [00:00<?, ?ex/s]

  0%|          | 0/632 [00:00<?, ?ex/s]

よそう そと の じたい に でんりょく かいしゃ も ちょっぴり こんわく きみ だ 


In [14]:



tokenizer = Wav2Vec2CTCTokenizer(vocabPath, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# tokenizer.push_to_hub("ttop324/wav2vec2-live-japanese")

In [15]:
def prepare_dataset(batch):    
    batch["input_values"] = processor(batch["speech"], sampling_rate=16000).input_values
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

datasets_train = datasets_train.map(prepare_dataset, batch_size=8, num_proc=12, batched=True,remove_columns=datasets_train.column_names)
datasets_test = datasets_test.map(prepare_dataset, batch_size=8, num_proc=12, batched=True,remove_columns=datasets_test.column_names)


/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists

In [16]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer

wer_metric = load_metric("wer")


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}
    

    

    

In [17]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

# model = Wav2Vec2ForCTC.from_pretrained("wav2vec2_large_xlsr_japanese_hiragana_1028")
model.freeze_feature_extractor()


training_args = TrainingArguments(
  output_dir=model_temp_output_dir,  
  group_by_length=True,
  per_device_train_batch_size=3,
  gradient_accumulation_steps=2,
  per_device_eval_batch_size=2,
  num_train_epochs=50,
  fp16=True,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=1,
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=datasets_train,
    eval_dataset=datasets_test,
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/ttop324/wav2vec2-live-japanese into local empty directory.


Download file pytorch_model.bin:   0%|          | 4.24k/1.18G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.55k/2.55k [00:00<?, ?B/s]

Clean file training_args.bin:  39%|###9      | 1.00k/2.55k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Using amp fp16 backend


In [ ]:
%%capture var
trainer.train()





In [ ]:
var.show()

/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630836880/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/transformers/trainer.py:1349: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.


Epoch,Training Loss,Validation Loss,Wer
0,1.165800,0.750287,0.390629
1,0.484100,0.616279,0.360688
2,0.425200,0.638450,0.345994
3,0.385800,0.647338,0.338231
4,0.355400,0.619789,0.334904
5,0.332800,0.611513,0.318686
6,0.307500,0.611265,0.312587
7,0.297400,0.621610,0.311894
8,0.272400,0.628998,0.294566
9,0.263800,0.608061,0.306765


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

In [19]:
processor.save_pretrained("wav2vec2_large_xlsr_japanese_hiragana_1028")
model.save_pretrained("wav2vec2_large_xlsr_japanese_hiragana_1028")

Configuration saved in wav2vec2_large_xlsr_japanese_hiragana_1028/preprocessor_config.json
tokenizer config file saved in wav2vec2_large_xlsr_japanese_hiragana_1028/tokenizer_config.json
Special tokens file saved in wav2vec2_large_xlsr_japanese_hiragana_1028/special_tokens_map.json
Configuration saved in wav2vec2_large_xlsr_japanese_hiragana_1028/config.json
Model weights saved in wav2vec2_large_xlsr_japanese_hiragana_1028/pytorch_model.bin


In [18]:
trainer.push_to_hub("ttop32/wav2vec2-live-japanese-translator")

Saving model checkpoint to ttop324/wav2vec2-live-japanese
Configuration saved in ttop324/wav2vec2-live-japanese/config.json
Model weights saved in ttop324/wav2vec2-live-japanese/pytorch_model.bin
Configuration saved in ttop324/wav2vec2-live-japanese/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 32.0k/1.18G [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.80k/2.80k [00:00<?, ?B/s]

To https://huggingface.co/ttop324/wav2vec2-live-japanese
   835f696..45ab74b  main -> main

Dropping the following result as it does not have all the necessary field:
{}
To https://huggingface.co/ttop324/wav2vec2-live-japanese
   45ab74b..9e975ee  main -> main



'https://huggingface.co/ttop324/wav2vec2-live-japanese/commit/45ab74b921bf77ea9810f14dce80c767e3b7bdae'